# FACIAL RECOGNITION 

In [ ]:
import cv2
import face_recognition
import numpy as np
import os
import sqlite3

def create_database(conn):
    # Create a table to store voter data
    conn.execute("CREATE TABLE IF NOT EXISTS VOTERID (ID INTEGER PRIMARY KEY, Name TEXT NOT NULL, Age INT NOT NULL, Gender TEXT NOT NULL, CH INT DEFAULT 0 NOT NULL, ImagePath TEXT NOT NULL, Encoding BLOB)")

def add_face_data(conn, voter_id, name, age, gender, image_path, encoding):
    # Insert the voter data, image path, and facial encoding into the database
    conn.execute("INSERT INTO VOTERID (ID, Name, Age, Gender, CH, ImagePath, Encoding) VALUES (?, ?, ?, ?, ?, ?, ?)", (voter_id, name, age, gender, 0, image_path, encoding))
    conn.commit()

def compare_face(conn, encoding):
    # Retrieve all stored face encodings from the database
    cursor = conn.execute("SELECT Encoding FROM VOTERID")
    encodings = [np.frombuffer(row[0], dtype=np.float64) for row in cursor]

    # Compare the new encoding with the stored encodings
    results = face_recognition.compare_faces(encodings, encoding)

    # Retrieve the names of the matched faces
    cursor = conn.execute("SELECT Name FROM VOTERID")
    names = [row[0] for row in cursor]

    # Return the names of the matched faces
    matched_names = [name for name, result in zip(names, results) if result]
    return matched_names

# Connect to the SQLite database
conn = sqlite3.connect('people.db')

# Create the database table
create_database(conn)

# Prompt for the voter's ID

# Capture photo from webcam
video_capture = cv2.VideoCapture(0)
ret, frame = video_capture.read()

# Define the path to save the captured image
name = input("Enter the voter's name: ")
existing_image_paths = [f for f in os.listdir('C:/IOT') if f.startswith(name)]
image_path = f"C:/IOT/{name}_face1.jpg"

# Update the image path if the name already exists in the folder
if existing_image_paths:
    last_image_index = max([int(f.split('_')[1].split('.')[0][4:]) for f in existing_image_paths])
    new_image_index = last_image_index + 1
    image_path = f"C:/IOT/{name}_face{new_image_index}.jpg"

# Save the captured image
cv2.imwrite(image_path, frame)

# Convert the image from BGR color (used by OpenCV) to RGB color (used by face_recognition)
rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

# Detect faces in the frame
face_locations = face_recognition.face_locations(rgb_frame)
if len(face_locations) == 0:
    print("No face detected.")
else:
    # Encode the first detected face
    face_encoding = face_recognition.face_encodings(rgb_frame, face_locations)[0]

    # Add voter data, image path, and face encoding to the database
    voter_id = int(input("Enter the voter's ID: "))
    age = int(input("Enter the voter's age: "))
    gender = input("Enter the voter's gender: ")
    add_face_data(conn, voter_id, name, age, gender, image_path, face_encoding)

    print("Face data added to the database.")

# Release the webcam and close the database connection
video_capture.release()
conn.close()

In [1]:
import cv2
import face_recognition
import numpy as np
import sqlite3

def compare_face(conn, encoding):
    # Retrieve all stored face encodings and names from the database
    cursor = conn.execute("SELECT Encoding, Name FROM VOTERID")
    rows = cursor.fetchall()
    encodings = [np.frombuffer(row[0], dtype=np.float64) for row in rows]
    names = [row[1] for row in rows]

    # Compare the new encoding with the stored encodings
    results = face_recognition.compare_faces(encodings, encoding)

    # Retrieve the names of the matched faces
    matched_names = [name for name, result in zip(names, results) if result]
    matched_data = []

    # Retrieve the data (except the last column) for the matched names
    for name in matched_names:
        cursor = conn.execute("SELECT * FROM VOTERID WHERE Name = ?", (name,))
        data = cursor.fetchone()
        matched_data.append(data[:-1])  # Exclude the last column

    return matched_data

# Connect to the SQLite database
conn = sqlite3.connect('people.db')

# Capture photo from webcam
video_capture = cv2.VideoCapture(0)
ret, frame = video_capture.read()

# Convert the image from BGR color (used by OpenCV) to RGB color (used by face_recognition)
rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

# Detect faces in the frame
face_locations = face_recognition.face_locations(rgb_frame)
if len(face_locations) == 0:
    print("No face detected.")
else:
    # Encode the first detected face
    face_encoding = face_recognition.face_encodings(rgb_frame, face_locations)[0]

    # Perform face recognition
    matched_data = compare_face(conn, face_encoding)

    if len(matched_data) == 0:
        print("Face not recognized.")
    else:
        print("Matched person(s):")
        for data in matched_data:
            print(data)

# Release the webcam and close the database connection
video_capture.release()
conn.close()


Matched person(s):
(1, 'CHANDU S GOWDA', 20, 'M', 0, 'C:/C/CHANDU S GOWDA_face1.jpg')
(2, 'CB', 21, 'M', 0, 'C:/C/CB_face2.jpg')
